<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#⭐-STAR-⭐" data-toc-modified-id="⭐-STAR-⭐-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>⭐ STAR ⭐</a></span><ul class="toc-item"><li><span><a href="#🔎-Situation-🔍" data-toc-modified-id="🔎-Situation-🔍-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>🔎 Situation 🔍</a></span></li><li><span><a href="#⚠️-Threat-⚠️" data-toc-modified-id="⚠️-Threat-⚠️-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>⚠️ Threat ⚠️</a></span></li><li><span><a href="#⚒️-Action-⚒️" data-toc-modified-id="⚒️-Action-⚒️-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>⚒️ Action ⚒️</a></span></li><li><span><a href="#📋-Result-📋" data-toc-modified-id="📋-Result-📋-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>📋 Result 📋</a></span></li></ul></li><li><span><a href="#Workflow-2:-Don't-Drop-Labels" data-toc-modified-id="Workflow-2:-Don't-Drop-Labels-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Workflow 2: Don't Drop Labels</a></span></li><li><span><a href="#⚠️-⚒️-TODO:-⚒️-⚠️" data-toc-modified-id="⚠️-⚒️-TODO:-⚒️-⚠️-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>⚠️ ⚒️ TODO: ⚒️ ⚠️</a></span></li><li><span><a href="#Workflow-1:-Dropping-Unused-Labels" data-toc-modified-id="Workflow-1:-Dropping-Unused-Labels-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Workflow 1: Dropping Unused Labels</a></span></li><li><span><a href="#⚠️-CONDENSE-W/-UPPER-WF-⚠️" data-toc-modified-id="⚠️-CONDENSE-W/-UPPER-WF-⚠️-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>⚠️ CONDENSE W/ UPPER WF ⚠️</a></span></li></ul></div>

# Intro

# ⭐ STAR ⭐

## 🔎 Situation 🔍

---

* NA uses DRR summary reports to post revenue in LS
* I use report packet to review for DRR

---

## ⚠️ Threat ⚠️

---

* **Pro:** clearly provide total sum amounts for audit to post directly

* **Cons:**
    * "Food" = "Food" + "Other"
    * Discounts are ambiguous
        * FBC vs. Assoc vs. others
    * *LOTS* of unused rows/labels create visual confusion
    * Not always clear to which XAC to post revenue/tax/tips
    * Unusual/new/unused categories are not always accounted for

All issues result in manual review of all transactions during the Daily reporting process.

---
    

## ⚒️ Action ⚒️

---

**Create workflow to:**

* *Read existing DD reports*


* *Clean the report to isolate relevant details*


* *Report any irregularities:*
    * Any discounts or charges?
    * Any new/non-standard MOPs?
    * Report exact amounts to post to each LS XAC
        * See below

---

---

**End Goal:**

---

> **Post these amounts in LightSpeed:**

<!-- | |Dept | Rain 903 | IRD  | CC | Bar| 
| - | - | - | - | - | - |
|**Sub-Dept** | |  |  |  |
|Food | 1.00 | 1.00 | 1.00 | 1.00 |
|Liquor | 2.00 | 2.00 | 2.00 | 2.00 
|Tax | 3.00 | 3.00 | 3.00 | 3.00 
|Gratuity | 4.00 | 4.00 | 4.00 | 4.00  -->

|Dept| Food | Liquor | Tax  | Gratuity | 
| -  | -    | -      | -    | -        |
| DR | 1.00 | 1.00   | 1.00 | 1.00     |
| RS | 4.00 | 4.00   | 4.00 | 4.00     |
| CC | 1.00 | 1.00   | 1.00 | 1.00     |
| LB | 4.00 | 4.00   | 4.00 | 4.00     |

---

 >**Investigate these amounts:**
 
 | Item            | Amount |
 | -               | - |
 | Discount        | 1.00 |
 | Charge          | 1.00 |
 | $2 Tip - Coupon | 1.00 |

---

## 📋 Result 📋

In [ ]:
## --- Importing Packages --- ##

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import bmcds.eda as eda

pd.set_option('display.max_rows', None)
pd.set_eng_float_format(accuracy=2, use_eng_prefix=True)
%matplotlib inline

In [ ]:
## --- Specify file path --- ##

file_path = './data/02_25_2022_pc.xls'

In [ ]:
## --- Read specific columns from Excel file --- ##

df = pd.read_excel(file_path, header = None, skiprows = 3,
                   usecols = 'A, B, D, F, H', 
                   names = ['Label', 'Lounge', 'Rain 903', 'In-Room Dining',
                            'Coffee Corner'])
df

In [ ]:
## --- How many rows have missing values, and how many? --- ##

df.isna().sum(axis=1).value_counts()

In [ ]:
## --- Filter out those rows with missing values --- ##

df_cleaned = df[df.isna().sum(axis=1) ==0]
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned

# Workflow 2: Don't Drop Labels

In [ ]:
## --- Create new DF to allow for later reuse --- ##

df_no_drp = df_cleaned.copy()
df_no_drp

In [ ]:
## --- Dropping blank rows --- ##

df_no_drp = df_no_drp[~df_no_drp.loc[:, "Label"].isin([' '])]
df_no_drp

In [ ]:
## --- Relabel and reset index with correct labels --- ##

df_no_drp = df_no_drp.rename(df_no_drp["Label"])
df_no_drp = df_no_drp.drop(columns = "Label")
df_no_drp

In [ ]:
## --- Review datatypes --- ##

df_no_drp.dtypes

In [ ]:
## --- Recast columns to "float" datatype & review --- ##

for col in df_no_drp.columns:
    df_no_drp[col] = pd.to_numeric(df_no_drp[col], downcast = 'float')

print(df_no_drp.dtypes)

display(df_no_drp)

In [ ]:
## --- Inspect results --- ##

df_no_drp.T.describe().T

In [ ]:
## --- Create new total column --- ##

df_no_drp['Total'] = df_no_drp.sum(axis=1)
df_no_drp

In [ ]:
## --- Filter rows without any activity --- ##

df_no_zero = df_no_drp[df_no_drp['Total'] > 0]
df_no_zero

# ⚠️ ⚒️ TODO: ⚒️ ⚠️

1. Filter non-zero dataframe into common/uncommon categories
2. Create DF w/ LS XAC for normal use
3. Create DF w/ unusual cats w/ prompt to review

# Workflow 1: Dropping Unused Labels

---

**Dropping Labels**

> Currently, I ignore several sections of the report as they do not report any relevant details. However, **this may lead us to overlook any irregular data.**
>
> This work flow focused on the most commonly used section of the report, highlighting the details used for NA.
>
> While I prefer a more inclusive approach, I am keeping this work flow for reference.

---

In [ ]:
## --- Identify unique labels to determine which to drop --- ##

df_cleaned.loc[:, "Label"].unique()

In [ ]:
## --- Create list of labels to drop --- ##

drop_labels = ['Sales Subtotal', 'Total before tax', 'None', 'None', 
               'Tax Subtotal','Receipt Rounding','Total of check', ' ', 'Unadjusted receipts',
               'PaidIn - AR', 'PaidIn - Debitek','PaidIn - Gift Certificates',
               'No of Covers', 'No of Checks', 'Avg cover', 'Avg check',
               'Total Paid Outs', 'Server bank', 'Total tips',
               'Server credit card fees', 'Net tips', 'Net Cash',
               'Other receipts turned in', 'Server Drops', 'Turned in',
               'Restaurant credit card fees', 'Estimated deposit',
               'Beginning NRS Total', 'Total Receipts', 'Discounts',
               'Paid Ins','Gross Total', 'NRS Total']
drop_labels

In [ ]:
## --- Drop specified labels and save to new DF --- ##

df_new_idx =df_cleaned[~df_cleaned.loc[:, "Label"].isin(drop_labels)]
df_new_idx

In [ ]:
## --- Rename index with remaining labels --- ##

df_new_idx = df_new_idx.rename(df_cleaned["Label"])
df_new_idx = df_new_idx.drop(columns = "Label")
df_new_idx

# ⚠️ CONDENSE W/ UPPER WF ⚠️

In [ ]:
## --- Review remaining datatypes --- ##

df_new_idx.dtypes

In [ ]:
## --- Recast DT --- ##

for col in df_new_idx.columns:
    df_new_idx[col] = pd.to_numeric(df_new_idx[col], downcast = 'float')

df_new_idx.dtypes

In [ ]:
## --- Inspect results --- ##

df_new_idx.T.describe()

In [ ]:
## --- Create new total column --- ##

df_new_idx['Total'] = df_new_idx.sum(axis=1)
df_new_idx

In [ ]:
## --- Filter out zero sums --- ##

df_compressed = df_new_idx[df_new_idx['Total'] >0]
df_compressed

In [ ]:
df_compressed['Lounge']['Food']

In [ ]:
## --- Test print statements for logging purposes --- ##

for col in df_compressed.columns[:-1]:
    print(f"Post to {col} food: {df_compressed[col]['Food']+df_compressed[col]['Other']:.2f}")

In [ ]:
## --- Further testing for logging purposes --- ##

dept_list = ["Rain 903", "In Room Dining", "Coffee Corner", "Lobby Bar"]
sd_list = ["Food", "Liquor", "Tax", "Gratuity"]

for dept in dept_list:
    for sd in sd_list:
        print(dept+" - "+sd)